#  Day 15
TIL about implemnting backtracking

In [866]:
from intcode_machine import IntcodeComputer
from collections import defaultdict
from itertools import cycle
from random import choice

In [665]:
with open('data/day15.txt') as fn:
    program = fn.readlines()
    program = [int(i) for i in program[0].split(',')]

In [4943]:
class RepairDroid(IntcodeComputer):
    def __init__(self, bits):
        super().__init__(bits)
        self.position = (0,0)
        self.map = defaultdict(lambda: ' ')
        self.map[(0, 0)] = 'D'
        self.move_iter = cycle([1,4,2,3])
        self.move_iter_r = cycle([1,3,2,4])
        self.facing = 1
        self.last_move = 0
        self.repeated = False
        self.wall = False
        
    def make_move(self, direction):
        """north (1), south (2), west (3), and east (4)"""
        Delta = {1: (0, 1),
                 2: (0, -1),
                 3: (-1, 0),
                 4: (1, 0)}
        
        output = self.run(direction)
        #print(output)
        dx, dy = Delta[direction]
        tx, ty = self.position[0] + dx, self.position[1] + dy
        #print(tx, ty)
        if output == 0:
            self.map[(tx, ty)] = '#'
            self.wall = True
        elif output == 1:
            self.map[self.position] = '.'
            self.position = (tx, ty)
            if self.map[self.position] != ' ':
                self.repeated=True
            self.map[self.position] = 'D'
        elif output == 2:
            self.map[self.position] = '.'
            self.position = (tx, ty)
            self.map[(tx, ty)] = '*'
            
        self.last_move = output
        return output
            
    def print_map(self):
        k0s = [k[0] for k in self.map.keys()]
        k1s = [k[1] for k in self.map.keys()]
        xrange = min(k0s), max(k0s)
        yrange = min(k1s), max(k1s)
        map_out = ""
        for y in range(yrange[1], yrange[0]-1, -1):
            for x in range(xrange[0], xrange[1]+1):
                if (x==0) & (y==0):
                    map_out += "S"
                else:
                    map_out += self.map[(x,y)]
            map_out += "\n"
        print(map_out)
        
    def explore(self, print_map=False, hug_right=True):
        pair={1:4, 2:3, 3:1, 4:2}
        a1 = self.make_move(self.facing)
        if a1 == 2:
            print("!!!!")
            return 2
        a2 = self.make_move(pair[self.facing])
        if a2 == 2:
            print("!!!!")
            return 2
        
        # Hug the wall
        if a2 != 0:
            self.facing = next(self.move_iter) if hug_right else next(self.move_iter_r)
        if (a1 == 0) & hug_right:
            next(self.move_iter)
            next(self.move_iter)
            self.facing = next(self.move_iter)
        elif (a1==0) & ~hug_right:
            next(self.move_iter_r)
            next(self.move_iter_r)
            self.facing = next(self.move_iter_r)
        if print_map:
            self.print_map()
        return a1
    
    def n_moves(self):
        return len([v for k, v in self.map.items() if v == '.'])
        
            

In [4944]:
def search(directions):
    global droid

    droid = RepairDroid(program)
    
    for d in directions:
        r = droid.make_move(d)
        if r == 0:
            return False
        if droid.repeated:
            return False
        if r == 2:
            return True
    
    if search(directions+[1]) or search(directions+[2]) or search(directions+[3]) or search(directions+[4]):
        return True
    
    return False
    

In [4945]:
search([1])

True

In [4946]:
droid.print_map()

              .....        
              .   .        
    ...     ...   .        
    . .     .     .        
    . .......     .        
    .             .        
    .   ...       .        
    .   . .       .        
    ... . ...     . ...    
      . .   .     . . .    
      ...   . ... ... ...  
            . . .       .  
            . . ....... .  
            . .       . .  
            . .   ..... ...
            . .   .       .
            . ... .........
            .   .          
    .....   .   .......    
    .   .   .         .    
.....   S ... ...   ...    
.         .   . .   .      
. ...     ..... .   .      
. . .           .   .      
. . ...         ... .      
. .   .           . .      
. .   .           . ...    
. .   .           .   .    
... ...           .   .    
    .             .   .    
    .         ... .   ...  
    .         . . .     .  
    .     ..... . ..... ...
    .     .     .     .   .
    ...   . .....   ...   .
      .   . .       

In [4947]:
droid.n_moves()

304

In [4948]:
droid.position

(18, -18)

# Q2

In [5009]:
droid = RepairDroid(program)

In [5011]:
r=0
while r!=2:
    r=droid.explore(hug_right=True)
droid.print_map()

!!!!
 ### # ##### ######### ######### ####### 
#...#.#.....#.........#.........#.......#
#.#.#.#.#.###.#######.#.###.###.###.###.#
#.#...#.#.....#.#...#...#...#.#.#...#.#.#
#.###.#.#######.#.#.#####.###.#.#.###.#.#
#...#.#.#.......#.#.......#...#.#...#...#
#.#.#.#.#.#####.#.#.#######.#.#.#.#.###.#
#.#.#.#.#.#...#.#.#.#...#...#.#.#.#...#.#
 ##.#.#.#.#.#.###.###.#.###.#.#.#####.## 
#...#.#.#...#...#...#.#...#.#.#.#...#...#
#.## ##.#.## ##.###.#.###.#.###.#.#.###.#
#...#...#...#.....#...#.#.#...#...#...#.#
#.#.#.#####.#.###.#####.#.#.#.#######.#.#
#.#...#...#.#.#...#.....#.#.#.......#.#.#
#.#####.#.###.#.#######.#.#.#######.#.#.#
#...#...#.....#.#.....#.#.#.#.#.....#...#
 ##.#.#########.#.###.#.#.#.#.#.#######.#
#...#.#.....#.....#.....#.#...#.........#
#.###.#.#####.#########.#.###.########## 
#.#...#.........#.....#.#.#.........#...#
#.###.#.#########.###.###.#########.#.## 
#.#...#.....#.....#.#S#...#...#.#...#...#
#.#.## ####.#.#####.###.###.#.#.#.#####.#
#.#...#...#.#.#...#...#.....#

In [5012]:
import copy
def backtrack(main_droid):
    main_droid = copy.deepcopy(main_droid)
    droids = [copy.deepcopy(main_droid)]
    for _ in range(1000):
        next_droids = []
        for droid in droids:
            for i in range(1,5):
                newdroid = copy.deepcopy(droid)
                res = newdroid.make_move(i)

                if (res==1) & (main_droid.map[newdroid.position]!='O'):
                    next_droids.append(newdroid)
                    main_droid.map[newdroid.position] = 'O'
        if len(next_droids) == 0:
            print(_)
            break
        
        droids = next_droids.copy()
    return main_droid
        
        

In [5013]:
D = backtrack(droid)

310


In [5014]:
droid.print_map()

 ### # ##### ######### ######### ####### 
#OOO#O#OOOOO#OOOOOOOOO#OOOOOOOOO#OOOOOOO#
#O#O#O#O#O###O#######O#O###O###O###O###O#
#O#OOO#O#OOOOO#O#OOO#OOO#OOO#O#O#OOO#O#O#
#O###O#O#######O#O#O#####O###O#O#O###O#O#
#OOO#O#O#OOOOOOO#O#OOOOOOO#OOO#O#OOO#OOO#
#O#O#O#O#O#####O#O#O#######O#O#O#O#O###O#
#O#O#O#O#O#OOO#O#O#O#OOO#OOO#O#O#O#OOO#O#
 ##O#O#O#O#O#O###O###O#O###O#O#O#####O## 
#OOO#O#O#OOO#OOO#OOO#O#OOO#O#O#O#OOO#OOO#
#O## ##O#O## ##O###O#O###O#O###O#O#O###O#
#OOO#OOO#OOO#OOOOO#OOO#O#O#OOO#OOO#OOO#O#
#O#O#O#####O#O###O#####O#O#O#O#######O#O#
#O#OOO#OOO#O#O#OOO#OOOOO#O#O#OOOOOOO#O#O#
#O#####O#O###O#O#######O#O#O#######O#O#O#
#OOO#OOO#OOOOO#O#OOOOO#O#O#O#O#OOOOO#OOO#
 ##O#O#########O#O###O#O#O#O#O#O#######O#
#OOO#O#OOOOO#OOOOO#OOOOO#O#OOO#OOOOOOOOO#
#O###O#O#####O#########O#O###O########## 
#O#OOO#OOOOOOOOO#OOOOO#O#O#OOOOOOOOO#OOO#
#O###O#O#########O###O###O#########O#O## 
#O#OOO#OOOOO#OOOOO#O#S#OOO#OOO#O#OOO#OOO#
#O#O## ####O#O#####O###O###O#O#O#O#####O#
#O#OOO#OOO#O#O#OOO#OOO#OOOOO#O#OOO